# Step 1: Install Dependencies

In [6]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset as TorchDataset
import random
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoModelForSeq2SeqLM,
    DataCollatorWithPadding
)
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, accuracy_score
from tqdm.auto import tqdm
import gc
import warnings

# =============================================================================
# 0. A: CHEMINS DES FICHIERS FOURNIS (MIS À JOUR POUR TÂCHE 1)
# =============================================================================
EN_TRAIN_FILE_PATH = '/content/eng_train_1.csv' # MIS À JOUR
SW_TRAIN_FILE_PATH = '/content/swa_train_1.csv' # MIS À JOUR
EN_TEST_FILE_PATH = '/content/eng_test_1.csv'   # MIS À JOUR

# Le fichier de soumission précédent (si nécessaire, mais non utilisé dans ce pipeline)
PREVIOUS_SUBMISSION_PATH = '/content/pred_eng.csv'

# =============================================================================
# 0. B: CONFIGURATION ET HYPERPARAMÈTRES
# =============================================================================
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
gc.collect()
torch.cuda.empty_cache()
warnings.filterwarnings('ignore')

# CONFIGURATION DU MODÈLE REMBERT (Classification)
CONFIG_MODEL = {
    'model_name': 'google/rembert',
    'max_length': 128,
    'k_folds': 5,
    'batch_size': 8,
    'gradient_accumulation_steps': 2,
    'learning_rate': 2e-5,
    'num_epochs': 5,
    'output_dir_base': './rembert-polarization-en-kfold-TASK1-AUGMENTED',
    'seed': SEED
}

# CONFIGURATION DE L'AUGMENTATION (Traduction NLLB)
CONFIG_AUG = {
    'nllb_model': "facebook/nllb-200-distilled-600M",
    'src_lang_sw': "swh_Latn",
    'tgt_lang_en': "eng_Latn",
    'batch_size': 32
}

# Définir le device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️ Device: {device} | 🎯 Tâche: Polarisation Binaire (Macro F1)")

# =============================================================================
# 0. C: CLASSES ET FONCTIONS DE BASE (Inchangées)
# =============================================================================
# (Définitions de PolarizationDataset, CustomTrainer, compute_metrics sont supposées être ici)
# ...

🖥️ Device: cuda | 🎯 Tâche: Polarisation Binaire (Macro F1)


In [7]:
# =============================================================================
# 1. DATA AUGMENTATION (SW Polarized -> EN)
# =============================================================================
def augment_polarized_data_binary(swa_file, config):
    print("\n--- 1. DATA AUGMENTATION (SW Polarized -> EN) ---")

    df_swa = pd.read_csv(swa_file)
    df_swa.columns = df_swa.columns.str.strip()
    text_col = next((c for c in df_swa.columns if c.lower() in ['text', 'text']), df_swa.columns[1])

    # **Filtrer UNIQUEMENT les exemples classés comme 'Polarized' (label=1)**
    df_polarized = df_swa[df_swa['label'] == 1].copy()

    # ... (le reste de la fonction de traduction est inchangé) ...
    # 2. Charger le modèle NLLB
    tokenizer_nllb = AutoTokenizer.from_pretrained(config['nllb_model'])
    model_nllb = AutoModelForSeq2SeqLM.from_pretrained(config['nllb_model']).to(device)

    # 3. Traduction
    texts = df_polarized[text_col].astype(str).tolist()
    results = []
    tokenizer_nllb.src_lang = config['src_lang_sw']

    for i in tqdm(range(0, len(texts), config['batch_size']), desc="Translating SW->EN"):
        batch_texts = texts[i : i + config['batch_size']]
        inputs = tokenizer_nllb(batch_texts, return_tensors="pt", padding=True, truncation=True).to(device)
        with torch.no_grad():
            translated_tokens = model_nllb.generate(**inputs, forced_bos_token_id=tokenizer_nllb.convert_tokens_to_ids(config['tgt_lang_en']), max_length=128)
        decoded = tokenizer_nllb.batch_decode(translated_tokens, skip_special_tokens=True)
        results.extend(decoded)

    # 4. Nettoyage et création du DataFrame augmenté
    del model_nllb
    gc.collect()
    torch.cuda.empty_cache()

    df_aug = df_polarized[['label']].copy()
    df_aug[text_col] = results

    return df_aug

# Exécution de l'augmentation et fusion
df_en_original = pd.read_csv(EN_TRAIN_FILE_PATH)
df_en_original.columns = df_en_original.columns.str.strip()

df_aug_en = augment_polarized_data_binary(SW_TRAIN_FILE_PATH, CONFIG_AUG)

if df_aug_en is not None:
    # Assurer la cohérence des colonnes
    text_col_orig = next((c for c in df_en_original.columns if c.lower() in ['text', 'text']), df_en_original.columns[1])
    df_aug_en = df_aug_en.rename(columns={df_aug_en.columns[-1]: text_col_orig})
    df_en_final = pd.concat([df_en_original, df_aug_en], ignore_index=True)
else:
    df_en_final = df_en_original

print(f"\n✅ Total Anglais (Original + Augmentation): {len(df_en_final):,} échantillons.")
print(f"  Polarized (label=1) count: {df_en_final['label'].sum()}")


--- 1. DATA AUGMENTATION (SW Polarized -> EN) ---


KeyError: 'label'

In [ ]:
# =============================================================================
# 2. K-FOLD TRAINING SUR LE DATASET AUGMENTÉ (RemBERT)
# =============================================================================
def run_k_fold_training_augmented(df_full, config):
    # ... (Le corps de la fonction est inchangé par rapport à la réponse précédente) ...
    print(f"\n{'='*70}")
    print(f"🚀 STARTING K-FOLD TRAINING on AUGMENTED DATA (RemBERT)")
    print(f"{'='*70}")

    texts_all = df_full['text'].tolist()
    labels_all = df_full['label'].values

    skf = StratifiedKFold(n_splits=config['k_folds'], shuffle=True, random_state=config['seed'])
    all_fold_metrics = []
    tokenizer = AutoTokenizer.from_pretrained(config['model_name'])

    for fold, (train_index, val_index) in enumerate(skf.split(texts_all, labels_all)):
        # ... (Logique de Split, Poids de Classe, et chargement du Modèle RemBERT) ...
        train_labels = labels_all[train_index]
        weight_0 = len(train_labels) / (2.0 * np.sum(train_labels == 0))
        weight_1 = len(train_labels) / (2.0 * np.sum(train_labels == 1))
        class_weights = torch.tensor([weight_0, weight_1], dtype=torch.float32)

        train_dataset = PolarizationDataset([texts_all[i] for i in train_index], [labels_all[i] for i in train_index], tokenizer, config['max_length'])
        val_dataset = PolarizationDataset([texts_all[i] for i in val_index], [labels_all[i] for i in val_index], tokenizer, config['max_length'])

        gc.collect()
        torch.cuda.empty_cache()

        model = AutoModelForSequenceClassification.from_pretrained(config['model_name'], num_labels=2)

        output_dir_fold = f"{config['output_dir_base']}/fold_{fold+1}"
        training_args = TrainingArguments(
            output_dir=output_dir_fold, num_train_epochs=config['num_epochs'], per_device_train_batch_size=config['batch_size'],
            per_device_eval_batch_size=config['batch_size'], learning_rate=config['learning_rate'], warmup_ratio=config['warmup_ratio'],
            weight_decay=0.01, eval_strategy="epoch", save_strategy="epoch", save_total_limit=1, load_best_model_at_end=True,
            metric_for_best_model='macro_f1', greater_is_better=True, fp16=torch.cuda.is_available(),
            gradient_accumulation_steps=config['gradient_accumulation_steps'], report_to="none", seed=config['seed'],
        )

        trainer = CustomTrainer(
            model=model, args=training_args, train_dataset=train_dataset, eval_dataset=val_dataset,
            tokenizer=tokenizer, compute_metrics=compute_metrics, class_weights=class_weights,
            data_collator=DataCollatorWithPadding(tokenizer=tokenizer)
        )

        trainer.train()
        eval_results = trainer.evaluate()
        print(f"  Macro F1 (Fold {fold+1}): {eval_results['eval_macro_f1']:.4f}")
        trainer.save_model(output_dir_fold)

    print(f"\n{'='*70}")
    print("K-FOLD TRAINING COMPLETED. Models saved.")

    # Note: Vous devriez vérifier ici si le F1 Macro est > 0.7566 !

run_k_fold_training_augmented(df_en_final, CONFIG_MODEL)

In [3]:
# =============================================================================
# 3. PRÉDICTION FINALE (Vote d'Ensemble K-Fold)
# =============================================================================
def predict_ensemble_voting(test_file_path, model_path_base, k_folds, max_length):
    print("\n--- 3. PRÉDICTION PAR VOTE D'ENSEMBLE ---")

    df_test = pd.read_csv(test_file_path)
    df_test.columns = df_test.columns.str.strip()
    id_col = next((c for c in df_test.columns if c.lower() in ['id', 'id']), df_test.columns[0])
    text_col = next((c for c in df_test.columns if c.lower() in ['text', 'text']), df_test.columns[1])
    test_texts = df_test[text_col].astype(str).tolist()

    tokenizer = AutoTokenizer.from_pretrained(f"{model_path_base}1")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    all_fold_predictions = np.zeros((len(test_texts), k_folds), dtype=int)

    for fold in range(k_folds):
        fold_model_path = f"{model_path_base}{fold+1}"
        model = AutoModelForSequenceClassification.from_pretrained(fold_model_path)
        model.eval()
        model.to(device)

        fold_predictions = []
        with torch.no_grad():
            for i in tqdm(range(0, len(test_texts), 64), desc=f"Predicting Fold {fold+1}"):
                batch_texts = test_texts[i:i + 64]
                encodings = tokenizer(batch_texts, max_length=max_length, padding='max_length', truncation=True, return_tensors='pt').to(device)
                outputs = model(**encodings)
                predictions = torch.argmax(outputs.logits, dim=-1).cpu().numpy()
                fold_predictions.extend(predictions)

        all_fold_predictions[:, fold] = np.array(fold_predictions)
        del model
        gc.collect()
        torch.cuda.empty_cache()

    # Vote Majoritaire
    ensemble_votes = np.sum(all_fold_predictions, axis=1)
    final_predictions = (ensemble_votes > (k_folds / 2)).astype(int)

    # Création du fichier de soumission
    submission_df = pd.DataFrame()
    submission_df[id_col] = df_test[id_col]
    submission_df['label'] = final_predictions

    output_filename = 'polarization_english_augmented_submission.csv'
    submission_df.to_csv(output_filename, index=False)

    print(f"\n✅ Fichier de soumission FINAL créé : {output_filename}")
    return submission_df

# Exécution de la prédiction
submission_task1 = predict_ensemble_voting(
    EN_TEST_FILE_PATH,
    CONFIG_MODEL['output_dir_base'] + '/fold_',
    CONFIG_MODEL['k_folds'],
    CONFIG_MODEL['max_length']
)


--- 3. PRÉDICTION PAR VOTE D'ENSEMBLE ---


HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': './rembert-polarization-en-kfold-TASK1-AUGMENTED/fold_1'. Use `repo_type` argument if needed.